<a href="https://cognitiveclass.ai"><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork/images/IDSN-logo.png" width="400"> </a>

# Convolutional Neural Networks with Keras

Estimated time needed **30** mins


In this lab, we will learn how to use the Keras library to build convolutional neural networks. We will also use the popular MNIST dataset and we will compare our results to using a conventional neural network.


## Objectives for this Notebook    
* How to use the Keras library to build convolutional neural networks
* Convolutional neural network with one set of convolutional and pooling layers
* Convolutional neural network with two sets of convolutional and pooling layers



## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
      
1. <a href="#Import-Keras-and-Packages">Import Keras and Packages</a>   
2. <a href="#Convolutional-Neural-Network-with-One-Set-of-Convolutional-and-Pooling-Layers">Convolutional Neural Network with One Set of Convolutional and Pooling Layers</a>  
3. <a href="#Convolutional-Neural-Network-with-Two-Sets-of-Convolutional-and-Pooling-Layers">Convolutional Neural Network with Two Sets of Convolutional and Pooling Layers</a>  

</font>
</div>


### Install the necessary libraries


Let's start by installing the keras libraries and the packages that we would need to build a neural network.


In [1]:
# All Libraries required for this lab are listed below. The libraries need to be installed on Skills Network Labs. 
# If you run this notebook on a different environment, e.g. your desktop, you may want to install these.
!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow_cpu==2.18.0
!pip install matplotlib==3.9.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 159.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 144.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 MB 43.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 47.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 148.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5

#### Suppress the tensorflow warning messages
We use the following code to  suppress the warning messages due to use of CPU architechture for tensoflow.
You may want to **comment out** these lines if you are using the GPU architechture


In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Import Keras and Packages


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input
from keras.utils import to_categorical

When working with convolutional neural networks in particular, we will need additional packages.


In [4]:
from keras.layers import Conv2D # to add convolutional layers
from keras.layers import MaxPooling2D # to add pooling layers
from keras.layers import Flatten # to flatten data for fully connected layers

## Convolutional Neural Network with One Set of Convolutional and Pooling Layers


In [5]:
# import data
from keras.datasets import mnist

# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Let's normalize the pixel values to be between 0 and 1


In [6]:
X_train = X_train / 255 # normalize training data
X_test = X_test / 255 # normalize test data

Next, let's convert the target variable into binary categories


In [7]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1] # number of categories

Next, let's define a function that creates our model. Let's start with one set of convolutional and pooling layers.


In [8]:
def convolutional_model():
    
    # create model
    model = Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(16, (5, 5), strides=(1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

Finally, let's call the function to create the model, and then let's train it and evaluate it.


In [9]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
300/300 - 35s - 115ms/step - accuracy: 0.9206 - loss: 0.2896 - val_accuracy: 0.9723 - val_loss: 0.0932
Epoch 2/10
300/300 - 32s - 107ms/step - accuracy: 0.9768 - loss: 0.0796 - val_accuracy: 0.9789 - val_loss: 0.0666
Epoch 3/10
300/300 - 33s - 109ms/step - accuracy: 0.9831 - loss: 0.0563 - val_accuracy: 0.9822 - val_loss: 0.0542
Epoch 4/10
300/300 - 32s - 108ms/step - accuracy: 0.9873 - loss: 0.0424 - val_accuracy: 0.9849 - val_loss: 0.0454
Epoch 5/10
300/300 - 33s - 110ms/step - accuracy: 0.9894 - loss: 0.0349 - val_accuracy: 0.9855 - val_loss: 0.0412
Epoch 6/10
300/300 - 33s - 109ms/step - accuracy: 0.9912 - loss: 0.0284 - val_accuracy: 0.9846 - val_loss: 0.0423
Epoch 7/10
300/300 - 33s - 108ms/step - accuracy: 0.9926 - loss: 0.0239 - val_accuracy: 0.9880 - val_loss: 0.0367
Epoch 8/10
300/300 - 32s - 108ms/step - accuracy: 0.9943 - loss: 0.0191 - val_accuracy: 0.9880 - val_loss: 0.0356
Epoch 9/10
300/300 - 33s - 109ms/step - accuracy: 0.9955 - loss: 0.0158 - val_accuracy: 

------------------------------------------


## Convolutional Neural Network with Two Sets of Convolutional and Pooling Layers


Let's redefine our convolutional model so that it has two convolutional and pooling layers instead of just one layer of each.


In [10]:
def convolutional_model():
    
    # create model
    model = Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(8, (2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    return model

Now, let's call the function to create our new convolutional neural network, and then let's train it and evaluate it.


In [11]:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
300/300 - 36s - 120ms/step - accuracy: 0.8662 - loss: 0.4803 - val_accuracy: 0.9601 - val_loss: 0.1264
Epoch 2/10
300/300 - 34s - 113ms/step - accuracy: 0.9664 - loss: 0.1128 - val_accuracy: 0.9743 - val_loss: 0.0822
Epoch 3/10
300/300 - 34s - 115ms/step - accuracy: 0.9754 - loss: 0.0812 - val_accuracy: 0.9782 - val_loss: 0.0636
Epoch 4/10
300/300 - 34s - 114ms/step - accuracy: 0.9797 - loss: 0.0656 - val_accuracy: 0.9830 - val_loss: 0.0535
Epoch 5/10
300/300 - 34s - 114ms/step - accuracy: 0.9836 - loss: 0.0557 - val_accuracy: 0.9848 - val_loss: 0.0457
Epoch 6/10
300/300 - 34s - 113ms/step - accuracy: 0.9855 - loss: 0.0478 - val_accuracy: 0.9859 - val_loss: 0.0417
Epoch 7/10
300/300 - 34s - 113ms/step - accuracy: 0.9871 - loss: 0.0420 - val_accuracy: 0.9858 - val_loss: 0.0396
Epoch 8/10
300/300 - 34s - 114ms/step - accuracy: 0.9881 - loss: 0.0384 - val_accuracy: 0.9872 - val_loss: 0.0403
Epoch 9/10
300/300 - 34s - 114ms/step - accuracy: 0.9887 - loss: 0.0358 - val_accuracy: 

<h3>Practice Exercise 1</h3>


Let's see how batch size affects the time required and accuracy of the model training. 
For this, you can try to change batch_size to 1024 and check it's effect on accuracy


In [12]:
# Write your answer here

model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/10
59/59 - 34s - 580ms/step - accuracy: 0.6808 - loss: 1.1947 - val_accuracy: 0.8878 - val_loss: 0.3851
Epoch 2/10
59/59 - 32s - 539ms/step - accuracy: 0.9086 - loss: 0.3074 - val_accuracy: 0.9339 - val_loss: 0.2252
Epoch 3/10
59/59 - 32s - 549ms/step - accuracy: 0.9402 - loss: 0.2034 - val_accuracy: 0.9537 - val_loss: 0.1589
Epoch 4/10
59/59 - 32s - 540ms/step - accuracy: 0.9543 - loss: 0.1553 - val_accuracy: 0.9631 - val_loss: 0.1252
Epoch 5/10
59/59 - 41s - 702ms/step - accuracy: 0.9619 - loss: 0.1285 - val_accuracy: 0.9688 - val_loss: 0.1055
Epoch 6/10
59/59 - 32s - 548ms/step - accuracy: 0.9676 - loss: 0.1102 - val_accuracy: 0.9740 - val_loss: 0.0916
Epoch 7/10
59/59 - 32s - 547ms/step - accuracy: 0.9701 - loss: 0.0981 - val_accuracy: 0.9767 - val_loss: 0.0858
Epoch 8/10
59/59 - 32s - 545ms/step - accuracy: 0.9730 - loss: 0.0886 - val_accuracy: 0.9770 - val_loss: 0.0765
Epoch 9/10
59/59 - 32s - 543ms/step - accuracy: 0.9757 - loss: 0.0800 - val_accuracy: 0.9792 - val_loss:

Double-click <b>here</b> for the solution.

<!-- Your answer is below:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))
-->


<h3>Practice Exercise 2</h3>


Now, let's see how number of epochs  affect the time required and accuracy of the model training. 
For this, you can keep the batch_size=1024 and epochs=25 and check it's effect on accuracy


In [13]:
# Write your answer here

model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))

Epoch 1/25
59/59 - 35s - 589ms/step - accuracy: 0.6601 - loss: 1.2387 - val_accuracy: 0.8861 - val_loss: 0.3946
Epoch 2/25
59/59 - 33s - 555ms/step - accuracy: 0.9095 - loss: 0.3016 - val_accuracy: 0.9390 - val_loss: 0.2072
Epoch 3/25
59/59 - 33s - 553ms/step - accuracy: 0.9465 - loss: 0.1821 - val_accuracy: 0.9575 - val_loss: 0.1402
Epoch 4/25
59/59 - 33s - 560ms/step - accuracy: 0.9609 - loss: 0.1323 - val_accuracy: 0.9672 - val_loss: 0.1084
Epoch 5/25
59/59 - 32s - 547ms/step - accuracy: 0.9686 - loss: 0.1057 - val_accuracy: 0.9712 - val_loss: 0.0918
Epoch 6/25
59/59 - 33s - 551ms/step - accuracy: 0.9731 - loss: 0.0909 - val_accuracy: 0.9725 - val_loss: 0.0834
Epoch 7/25
59/59 - 33s - 552ms/step - accuracy: 0.9762 - loss: 0.0806 - val_accuracy: 0.9777 - val_loss: 0.0720
Epoch 8/25
59/59 - 33s - 552ms/step - accuracy: 0.9778 - loss: 0.0732 - val_accuracy: 0.9787 - val_loss: 0.0701
Epoch 9/25
59/59 - 32s - 550ms/step - accuracy: 0.9796 - loss: 0.0680 - val_accuracy: 0.9790 - val_loss:

Double-click <b>here</b> for the solution.

<!-- Your answer is below:
# build the model
model = convolutional_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1024, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {} \n Error: {}".format(scores[1], 100-scores[1]*100))


    -->


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!


<!--
## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2024-11-20  | 3.0  | Aman  |  Updated the library versions to current |
| 2020-09-21  | 2.0  | Srishti  |  Migrated Lab to Markdown and added to course repo in GitLab |



<hr>

## <h3 align="center"> © IBM Corporation. All rights reserved. <h3/>


## <h3 align="center"> &#169; IBM Corporation. All rights reserved. <h3/>

